In [1]:
from ground_data import GroundObservations
from LOCSS_data_structure import GaugeCollection
import importlib
import pandas as pd
import os
import datetime as dt
%load_ext autoreload
%autoreload 2

In [28]:
#Generate each lake in a folder
go=GroundObservations()
df_locss_podaac=go.get_locss('LOCSS', all_fds=True)

g_path='../data/locss_2_podaac/by_st'
f_exist=os.path.exists(g_path)
stations=df_locss_podaac['gauge_id'].unique()
df_locss_podaac.rename(columns={'name_x':'name_at_measurement', 'notes_x':'reading_notes', 'name_y':'lake_name',
                                       'notes_y':'installation_notes'}, inplace=True)
st_fields=['gauge_id','lake_name','date','height','is_bubble_level_okay','installation_notes', 'city','latitude','longitude','timezone',
           'unit','min_height','max_height','installation_date','initial_reading','reading_notes', 'updated_at']
df_locss_podaac=df_locss_podaac[st_fields].copy()
if not f_exist:
    os.makedirs(g_path)
for st in stations:
    st_path=g_path+'/'+st
    f_st_exist=os.path.exists(st_path)
    if not f_st_exist:
        os.makedirs(st_path)
    df=df_locss_podaac.loc[df_locss_podaac['gauge_id']==st]
    df.to_csv(st_path+'/LOCSS_'+st+'.csv')

In [25]:
df_locss_podaac.columns


Index(['gauge_id', 'lake_name', 'date', 'height', 'is_bubble_level_okay',
       'installation_notes', 'city', 'latitude', 'longitude', 'timezone',
       'unit', 'min_height', 'max_height', 'installation_date',
       'initial_reading', 'reading_notes', 'updated_at'],
      dtype='object')

In [33]:
#read with no merge for po.daac
dt_fields=['gauge_id','name_x','date','height','is_bubble_level_okay','notes_x']
st_fields=['gauge_id','name','city','latitude','longitude','timezone','unit','min_height','max_height','installation_date','initial_reading','notes', 'updated_at']

dir_ts='../data/readings_up_to_20220404.csv'
dir_loc='../data/gauges_up_to_20220404.csv'
id_fd='gauge_id'
date_fd='date'
time_fd='time'
height_fd='height'
df_locss=pd.read_csv(dir_ts)
df_coord_locss=pd.read_csv(dir_loc, sep=",")
gc=GaugeCollection()

df_locss_filtered=gc.filter_test_gages(df_locss,id_fd).copy()
df_locss_filtered=pd.merge(df_locss_filtered, df_coord_locss, on=id_fd)
df_locss_filtered=df_locss_filtered.loc[(df_locss_filtered[height_fd]>=df_locss_filtered['min_height'])&
                                        (df_locss_filtered[height_fd]<=df_locss_filtered['max_height'])].copy()
df_locss_filtered[date_fd]=pd.to_datetime((df_locss_filtered[date_fd].astype(str)+' '+df_locss_filtered[time_fd].astype(str)), 
                                                  format='%Y-%m-%d %H:%M:%S')


In [34]:
df_locss_filtered=df_locss_filtered[dt_fields]
df_locss_filtered.rename(columns={'name_x':'name', 'notes_x':'notes'}, inplace=True)
df_coord_locss=df_coord_locss[st_fields]
out_path='../data/locss_2_podaac/st_rd/'
f_exist=os.path.exists(out_path)
if not f_exist:
    os.makedirs(out_path)
df_locss_filtered.to_csv(out_path+'locss_readings_up_to_20220404.csv', index=False)
df_coord_locss.to_csv(out_path+'locss_gauges_up_to_20220404.csv', index=False)

In [67]:
#Test getting brasilian ts
import pandas as pd
import os


def _get_all_lakes(file_list, by=None):
    df=pd.DataFrame(file_list, columns=['file_name'])
    df_ex=df['file_name'].str.split('_', expand=True)
    
    df_ex=df_ex.rename(columns={0:'location',1:'lake_id',2:'name'})
    df_ex['name']=df_ex['name'].apply(lambda x:x[:-4])
    df_all=pd.concat((df,df_ex), axis=1)
    if by=='lake_id':
        return df_all['lake_id']
    elif by=='name':
        return df_all['name']
    else:
        return df_all
postfix='.xlsx'
path='../data/brasil/'
files = [f for f in os.listdir(path) if (os.path.isfile(os.path.join(path, f))&
                                                  (f.endswith(postfix)))]
files
start_st=3
end_st=8
st=[f[start_st:end_st] for f in files]
for s in st:
    print(files[st.index(s)])
    
# type(get_all_lakes(data, by='lake_id'))
# type(get_all_lakes(data, by=None))
# type(get_all_lakes(data, by='name'))
# _get_all_lakes(files, by='lake_id')
# df=_get_all_lakes(files, by=None)

# for index, row in df.iterrows():
#     print(row['file_name'])
#     df=pd.read_excel(path+row['file_name'])
#     break
# # [f[] for f in files] 
# df
# f[:-len(postfix)][-length_id:]
# _get_all_lakes(files, by='name')
# df_coord['lon_tx'].astype(float)/100

br_19069_MACHADINHO.xlsx
br_19126_XINGO.xlsx
br_19060_SEGREDO.xlsx


In [ ]:
#To test what is faster 

import xlrd
import timeit

path='/home/angelica/Documents/py_linux/pylocss/data/brasil/br_19126_XINGO.ods'
# xlrd.open_workbook(filename=path)
%timeit pd.read_excel(path, engine='odf')

In [ ]:
path='/home/angelica/Documents/py_linux/pylocss/data/brasil/br_19126_XINGO.xlsx'
# xlrd.open_workbook(filename=path)
%timeit pd.read_excel(path)

In [71]:
from ts_preprocesing import *
%load_ext autoreload
%autoreload 2


In [89]:
#test new function load altis with changes 
g_path='/home/angelica/Dropbox/apps/python_pychm/locss/data/alti_timeseries/'
altis_name='AlTiS_TimeSeries_Jason-3_GDR-F_0215_S4017'
altis_name='AlTiS_TimeSeries_Sentinel-3A_0749_S4016'
altis_date_fd = 'date'  # Same value for Sentinel-3A/B
altis_height_fd = 'ice1_ku_SurfHeight_alti_median'  # Same value for Sentinel-3A/B
nodataalti=-9999
wse_ref='g'
(altiyear, altimonth, altiday, altihour, altiminute, altiwelev)=load_altis(g_path+altis_name+'.csv', 
                                                                               altis_date_fd, altis_height_fd, nodataalti=-9999, wse_ref=wse_ref)

altiwelev

array([639.41776294, 638.79814716, 639.03307506, 639.08598951,
       638.9512785 , 639.22153111, 639.86431209, 639.76201283,
       639.94931077, 639.9821185 , 639.57211619, 639.297528  ,
       639.25073062, 639.17055127, 639.08187817, 639.40142945,
       639.39348567, 639.89583095, 639.7917532 , 640.25812196,
       640.10360696, 640.42202405, 640.42524257, 640.13159718,
       639.72194669, 639.1764573 , 639.00231116, 639.25268834,
       639.51002587, 639.60377963, 639.87170993, 640.49675565,
       640.03452703, 639.76819589, 640.20760798, 640.47443189,
       640.08513394, 639.50381148, 639.14197758, 638.95128786,
       639.06035578, 639.00788169, 639.46582925, 639.97037448,
       639.85536325, 640.67367134, 640.45245077, 640.07032121,
       640.21899268, 640.52368635, 639.96197048, 640.11167615,
       639.65527678, 639.47112128, 639.53081034, 639.4678867 ,
       640.09827893, 640.31512617, 640.54360892, 640.50777014,
       640.37066558, 640.49162851, 640.7388485 , 640.60

In [92]:
#test new function load altis with changes 
g_path='/home/angelica/Dropbox/apps/python_pychm/locss/data/alti_timeseries/'
altis_name='AlTiS_TimeSeries_Sentinel-3A_0749_S4016'
altis_date_fd = 'date'  # Same value for Sentinel-3A/B
altis_height_fd = 'ice1_ku_SurfHeight_alti_median'  # Same value for Sentinel-3A/B
nodataalti=-9999
wse_ref='e'
ncolgeoid='geoid_01_median'
(altiyear1, altimonth, altiday, altihour, altiminute, altiwelev1)=load_altis(g_path+altis_name+'.csv', 
                                                                            altis_date_fd, altis_height_fd, ncolgeoid=ncolgeoid,
                                                                            nodataalti=-9999, wse_ref=wse_ref)

altiwelev1

array([663.07981069, 662.47482143, 662.70496378, 662.74140667,
       662.61090978, 662.86109502, 663.50252798, 663.39862791,
       663.5459598 , 663.63459361, 663.22171309, 662.94419173,
       662.90517984, 662.82497422, 662.71601449, 663.08318062,
       663.0269616 , 663.57564589, 663.43465858, 663.93466536,
       663.75011991, 664.06734304, 664.06818937, 663.76005322,
       663.35692908, 662.83904057, 662.65802611, 662.89198135,
       663.16243485, 663.24401847, 663.51795276, 664.16366049,
       663.7086631 , 663.43072491, 663.87263624, 664.13096986,
       663.74833067, 663.14252579, 662.7969605 , 662.60973331,
       662.70915986, 662.64661454, 663.1199493 , 663.60987749,
       663.51366899, 664.33485192, 664.10550297, 663.71663475,
       663.86306733, 664.15908874, 663.61175306, 663.31361751,
       662.86337152, 662.66698005, 662.73149651, 662.6737999 ,
       663.30699027, 663.51799969, 663.74929344, 663.69564737,
       663.58061859, 663.69830568, 663.94073831, 663.81

In [100]:
#Making the correction from whole code

text=['AlTiS_TimeSeries_Sentinel-3A_0562_S4128', 'AlTiS_TimeSeries_Sentinel-3B_0026_N4814', 'AlTiS_TimeSeries_Jason-3_GDR-F_0215_S4017']
s=[f.find('Sentinel-3') for f in text]
for v in s:
    if v!=-1:
        print('get ellipsoide')
    else:
        print('get geoide')

get ellipsoide
get ellipsoide
get geoide


In [101]:
text[0].find('Sentinel')

17